# Sort and Slice Tutorial - An alternative to extended connectivity fingerprints

---


### Background

[Sort and Slice](https://doi.org/10.48550/arXiv.2403.17954) (SNS) was developed by a former OPIGlet, Markus, as a method for improving [Extended Connectivity Fingerprints (ECFPs)](https://doi.org/10.1021/ci100050t). ECFPs are a form of topological fingerprint which denote the absence and presence of circular substructures in a molecule. The steps in deriving an ECFP from a molecule are as follows:

1. Identifier assignment
    Each atom in the molecule is assigned an initial numerical identifier; this is typically generated by hashing a tuple of atomic properties called [Daylight atomic invariants](https://doi.org/10.1021/ci00062a008) into a 32-bit integer. These properties are:

    1. Number of non-hydrogen neighbours.
    2. Valence - number of neighbouring hydrogens.
    3. Atomic number.
    4. Atomic mass.
    5. Atomic charge.
    6. Number of hydrogen neighbours.
    7. Ring membership.*

    *Ring membership is an additional property that is often used but is not one of the original Daylight atomic invariants.

2. Iterative updating
    Each substructure identifier is updated by hashing the central identifier with the identifiers of its neighbours, i.e. performing a 1-hop aggregation. This processes is repeated upto a predetermined maximum radius.

3. Duplicate removal
    Duplicate substructures are identified and removed.

4. Folding into a smaller bit vector
    The 32-bit integers are used to produce a sparse vector, with 'on' bits at each substructure identifier. This sparse bit-vector is typically folded into smaller fixed length bit vector, typically of 1024 or 2048 bits. This final folding step often produces 'bit collisions', with the same bit being activated by different substructures which adds noise to the representation.

For background more on ECFPs, I recommend [this paper by Rogers and Hahn (2010)](https://doi.org/10.1021/ci100050t) which introduced ECFPs, Leo's blopig post on [topological fingerprints](https://www.blopig.com/blog/2022/06/exploring-topological-fingerprints-in-rdkit/), Markus' blopig post on [turning SMILES into ECFPs with RDKit](https://www.blopig.com/blog/2022/11/how-to-turn-a-smiles-string-into-an-extended-connectivity-fingerprint-using-rdkit/), and Greg Landrum's blog post on [fingerprint generators in RDKit](https://greglandrum.github.io/rdkit-blog/posts/2023-01-18-fingerprint-generator-tutorial.html).

SNS is a form of dataset dependent feature selection applied to ECFPs. Instead of folding the final sparse vector, SNS only retains substructures which occurred frequently in a previously observed dataset. Let's see how we might do this in Python.

In [146]:
# imports
import pandas as pd
from rdkit.Chem.rdFingerprintGenerator import AdditionalOutput, GetMorganGenerator
import numpy as np
from sklearn.model_selection import train_test_split
from rdkit import Chem
from tqdm import tqdm # progress bar
import jdc # allows dynamic class definition across cells
import warnings

### Data

First of all we need a dataset to test our code on. [QM9](https://doi.org/10.1038/sdata.2014.22) is a commonly used dataset small molecules with calculated quantum properties, which can be easily downloaded from [MoleculeNet](https://pubs.rsc.org/en/content/articlelanding/2018/sc/c7sc02664a) using pandas:

In [147]:
qm9 = 'https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/qm9.csv'
qm9 = pd.read_csv(qm9)
qm9.head()

,mol_id,smiles,A,B,C,mu,alpha,homo,lumo,gap,...,zpve,u0,u298,h298,g298,cv,u0_atom,u298_atom,h298_atom,g298_atom
0,gdb_1,C,157.71180,157.709970,157.706990,0.0000,13.21,-0.3877,0.1171,0.5048,...,0.044749,-40.478930,-40.476062,-40.475117,-40.498597,6.469,-395.999595,-398.643290,-401.014647,-372.471772
1,gdb_2,N,293.60975,293.541110,191.393970,1.6256,9.46,-0.2570,0.0829,0.3399,...,0.034358,-56.525887,-56.523026,-56.522082,-56.544961,6.316,-276.861363,-278.620271,-280.399259,-259.338802
2,gdb_3,O,799.58812,437.903860,282.945450,1.8511,6.31,-0.2928,0.0687,0.3615,...,0.021375,-76.404702,-76.401867,-76.400922,-76.422349,6.002,-213.087624,-213.974294,-215.159658,-201.407171
3,gdb_4,C#C,0.00000,35.610036,35.610036,0.0000,16.28,-0.2845,0.0506,0.3351,...,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,-385.501997,-387.237686,-389.016047,-365.800724
4,gdb_5,C#N,0.00000,44.593883,44.593883,2.8937,12.99,-0.3604,0.0191,0.3796,...,0.016601,-93.411888,-93.409370,-93.408425,-93.431246,6.278,-301.820534,-302.906752,-304.091489,-288.720028


Let's perform a train/test split and convert the SMILES strings into RDKit molecule objects:

In [148]:
train, test = train_test_split(qm9, test_size=0.4, random_state=42)

train_smiles = train['smiles'].to_list()
test_smiles = test['smiles'].to_list()

train_mols = [Chem.MolFromSmiles(smi) for smi in train_smiles]
test_mols = [Chem.MolFromSmiles(smi) for smi in test_smiles]

### Coding Sort and Slice

Now we have our data let's see how we can build a `SortAndSlice` class. There are several things our class needs to possess:
1. An RDKit Morgan Fingerprint generator for performing substructure identifier generation.
2. An RDKit AdditionalOutput object for obtaining substructure identifiers from the Morgan generator.
3. A method for obtaining all the substructure identifiers in the training dataset.
3. A dictionary for holding all the identifiers in the training dataset.
4. An encoding dictionary of a fixed length for mapping substructures to bits.
5. A method for converting new molecules to SNS bit vectors.

#### Class Constructor

Below is the constructor method of our `SortAndSlice` class, which takes a list of training data molecules, a Morgan Generator, the desired bit length, and a verbosity flag as input arguments. The constructor defines generator and AdditionalOutput instances as attributes of our class, before computing the identifiers in the dataset and the encoder.

In [149]:
class SortAndSlice:

    encoder = {}

    def __init__(
        self,
        data: list[Chem.Mol],
        generator: GetMorganGenerator = GetMorganGenerator(radius=2),
        bit_length: int = 2048,
        verbose: bool = False,
    ):
        self.generator = generator
        self.ao: AdditionalOutput = AdditionalOutput()
        self.ao.AllocateBitInfoMap()
        self.verbose = verbose

        self.identifiers = self.get_identifiers(data)
        self.set_encoder(bit_length)

    

#### Sort

With our constructor written, the next requirement is a method which calculates the frequency of substructure identifiers across our dataset and sorts identifiers. The method below, `get_identifiers`, does the following:
1. Creates an empty dictionary to tally substructure identifiers.
2. Iterates over each molecule in the input data. For each iteration:
    1. Calculates the sparse Morgan fingerprint of a molecule.
    2. Obtains all the substructure integer identifiers from the additional output object.
    3. For each integer, adds 1 to the total tally.
3. Sorts the identifiers by frequency.
4. Returns the identifiers as a dictionary, with identifiers as keys and tallies as values.


In [150]:
%%add_to SortAndSlice
def get_identifiers(self, data: list[Chem.Mol]) -> dict[str, int]:
    identifiers = {}
    pbar = tqdm(total=len(data), desc='Collecting identifiers', disable=not self.verbose)
    for mol in data:
        self.generator.GetSparseFingerprint(mol, additionalOutput=self.ao)
        bitmap = self.ao.GetBitInfoMap()
        for identifier in bitmap:
            count = identifiers.get(identifier, 0)
            identifiers[identifier] = count + 1
        pbar.update(1)
    pbar.close()
    identifiers = dict(sorted(identifiers.items(), key=lambda x: x[1], reverse=True))
    return identifiers


#### Slice

Having computed the identifiers in our dataset and sorted them by frequency, the slicing operation comes next. The method `set_encoder` takes an integer, `bit_length`, as an argument. The `SortAndSlice` encoder is then set by taking only the top `bit_length` substructures by frequency and labelling them with a rank, i.e. the identifiers dictionary is sliced and enumerated.

In [151]:
%%add_to SortAndSlice
def set_encoder(self, bit_length: int):
    if self.verbose:
        print(f'Setting bit length of encoder to a max of {bit_length}.')
    encoder = {}
    for i, k in enumerate(self.identifiers.keys()):
        if i >= bit_length:
            break
        encoder[k] = i

    self.encoder = encoder
    if len(encoder) < bit_length:
       warnings.warn(f'Encoder is only {len(encoder)} bits long.')

    if self.verbose:
        print(f'Encoder set to {len(encoder)} bits.')

#### Featurization

With an encoder, new molecules can now be encoded based on the molecules that have already been seen. The `encode` method, takes an rdkit molecule as an argument and performs the following:

1. Calculates the sparse morgan fingerprint of the molecule.
2. Obtains the 32-bit substructure identifiers in the molecule.
3. Generates an output vector of zeros that's the length of the encoder.
4. Iterates over the identifiers in the molecule. For each identifier:
    1. Checks if the identifier is in the encoder.
    2. If the identifier is encoded, the rank of the identifier is used to turn a bit in the output vector to 1.
5. Returns the SNS feature vector for the molecule.

In [152]:
%%add_to SortAndSlice
def encode(self, mol: Chem.Mol) -> np.ndarray:
    self.generator.GetSparseFingerprint(mol, additionalOutput=self.ao)
    bitmap = self.ao.GetBitInfoMap()
    out = np.zeros(len(self.encoder))
    for identifier in bitmap:
        if identifier in self.encoder:
            out[self.encoder[identifier]] = 1
    return out

#### Usage

Let's see how this `SortAndSlice` class can be used. Below a Morgan Fingerprint generator is defined. Next this generator and the QM9 training molecules are used as arguments for an instance of the SortAndSlice class:

In [153]:
gen = GetMorganGenerator(radius=2, includeChirality=True)
sns = SortAndSlice(train_mols, generator=gen, bit_length=2048, verbose=True)


Setting bit length of encoder to a max of 2048.
Encoder set to 2048 bits.


To use this `SortAndSlice` instance to featurize a set of molecules we can do the following:

In [154]:
test_sort_and_slice = [sns.encode(mol) for mol in test_mols]
len(test_sort_and_slice), test_sort_and_slice[0].shape, np.where(test_sort_and_slice[0])

(53554,
 (2048,),
 (array([   0,    1,    4,    5,   10,   20,   21,   22,   35,   53,   69,
          100,  629,  901, 1242]),))

And there you have it! A set of topological fingerprints without bit collisions!

For clarity, here's how the SortAndSlice class looks without cell breaks:

In [155]:
class SortAndSlice:
    """
    Class to sort and slice the output of a Morgan fingerprint generator.

    Args:
        data (list[Chem.Mol]): List of RDKit molecules.
        generator (GetMorganGenerator): RDKit fingerprint generator.
        bit_length (int): Length of the output vector.
        verbose (bool): Whether to print progress.

    Attributes:
        generator (GetMorganGenerator): RDKit fingerprint generator.
        ao (AdditionalOutput): RDKit fingerprint generator output.
        verbose (bool): Whether to print progress.
        identifiers (dict[str, int]): Dictionary of identifiers and counts.
        encoder (dict[str, int]): Dictionary of identifiers and indices.

    Methods:
        get_identifiers: Collects identifiers from data.
        set_encoder: Sets the encoder to a specific length.
        encode: Encodes a molecule into a binary vector.
    """
    encoder = {}

    def __init__(
        self,
        data: list[Chem.Mol],
        generator: GetMorganGenerator = GetMorganGenerator(radius=2),
        bit_length: int = 2048,
        verbose: bool = False,
    ):
        self.generator = generator
        self.ao: AdditionalOutput = AdditionalOutput()
        self.ao.AllocateBitInfoMap()
        self.verbose = verbose

        self.identifiers = self.get_identifiers(data)
        self.set_encoder(bit_length)

    def get_identifiers(self, data: list[Chem.Mol]) -> dict[str, int]:
        """
        Collects and sorts identifiers from molecule data.
        
        Args:
            data (list[Chem.Mol]): List of RDKit molecules.
            
        Returns:
            dict[str, int]: Dictionary of identifiers and counts.
        """
        identifiers = {}
        pbar = tqdm(total=len(data), desc='Collecting identifiers', disable=not self.verbose)
        for mol in data:
            self.generator.GetSparseFingerprint(mol, additionalOutput=self.ao)
            bitmap = self.ao.GetBitInfoMap()
            for identifier in bitmap:
                count = identifiers.get(identifier, 0)
                identifiers[identifier] = count + 1
            pbar.update(1)
        pbar.close()
        identifiers = dict(sorted(identifiers.items(), key=lambda x: x[1], reverse=True))
        return identifiers
    
    def set_encoder(self, bit_length: int):
        """
        Slices substructure identifiers to a specific length enumerates them.
        Sets the encoder attribute to the enumerated identifiers.

        Args:
            bit_length (int): Length of the output vector.
        """
        if self.verbose:
            print(f'Setting bit length of encoder to a max of {bit_length}.')
        encoder = {}
        for i, k in enumerate(self.identifiers.keys()):
            if i >= bit_length:
                break
            encoder[k] = i

        self.encoder = encoder
        if len(encoder) < bit_length:
            warnings.warn(f'Encoder is only {len(encoder)} bits long.')

        if self.verbose:
            print(f'Encoder set to {len(encoder)} bits.')

    def encode(self, mol: Chem.Mol) -> np.ndarray:
        """
        Encodes a molecule into a binary sort and slice vector.

        Args:
            mol (Chem.Mol): RDKit molecule.

        Returns:
            np.ndarray: Binary vector indicating substructure presence.
        """
        self.generator.GetSparseFingerprint(mol, additionalOutput=self.ao)
        bitmap = self.ao.GetBitInfoMap()
        out = np.zeros(len(self.encoder))
        for identifier in bitmap:
            if identifier in self.encoder:
                out[self.encoder[identifier]] = 1
        return out


### References:
1. Dablander, M. et al. (2024) Sort and Slice: A Simple and Superior Alternative to Hash-Based Folding for Extended-Connectivity Fingerprints. https://doi.org/10.48550/arXiv.2403.17954
2. Rogers, D. and Hahn, M. (2010) Extended-Connectivity Fingerprints. https://doi.org/10.1021/ci100050t
3. Weininger, D., Weininger, A., Weininger, J.L. (1989) SMILES. 2. Algorithm for generation of unique SMILES notation. https://doi.org/10.1021/ci00062a008
4. Klarner, L. (2022) Exploring Topological Fingerprints. https://www.blopig.com/blog/2022/06/exploring-topological-fingerprints-in-rdkit/
5. Dablander, M. (2022) How to turn a SMILES string into an extended-connectivity fingerprint using RDKit. https://www.blopig.com/blog/2022/11/how-to-turn-a-smiles-string-into-an-extended-connectivity-fingerprint-using-rdkit/
6. Landrum, G. (2023) FingerprintGenerator tutorial. https://greglandrum.github.io/rdkit-blog/posts/2023-01-18-fingerprint-generator-tutorial.html